In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report as report
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier,RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('sonar/sonar.all-data', header = None)
df=df.replace('R',int(0))
df=df.replace('M',int(1))
data=df.values
from sklearn.model_selection import train_test_split
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(data[:,0:60],data[:,60], test_size=test_size, random_state=seed)
y_test=y_test.astype(int)
y_train=y_train.astype(int)

In [3]:
lr_clf=LogisticRegression(max_iter=3000)
svm_clf=SVC()
rf_clf=RandomForestClassifier(
    n_estimators=21,
    criterion="gini",
    max_depth=4,
    random_state=0)

gbr = GradientBoostingClassifier(n_estimators=30000, max_depth=2, min_samples_split=2, learning_rate=0.01)
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=None, min_samples_split=20, min_samples_leaf=21),
                         algorithm="SAMME",
                         n_estimators=200, learning_rate=0.8)
xgb = XGBClassifier(learning_rate=0.01,max_depth=10)

In [4]:
lr_clf.fit(X_train,y_train)
svm_clf.fit(X_train,y_train)
rf_clf.fit(X_train,y_train)
gbr.fit(X_train, y_train.ravel())
bdt.fit(X_train,y_train)
eval_set = [(X_test, y_test)]
xgb.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="logloss", eval_set=eval_set, verbose=False)

E:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [5]:
lr_pred=lr_clf.predict(X_test)
svm_pred=svm_clf.predict(X_test)
rf_pred=rf_clf.predict(X_test)
gbr_pred = gbr.predict(X_test)
bdt_pred = bdt.predict(X_test)
xgb_pred = xgb.predict(X_test)

In [6]:
print('lr:\n',report(y_test,lr_pred))
print('svm:\n',report(y_test,svm_pred))
print('rf:\n',report(y_test, rf_pred))
print('GBDT:\n',report(y_test, gbr_pred))
print('AdaBoost:\n',report(y_test, bdt_pred))
print('XgBoost:\n',report(y_test, xgb_pred))

lr:
               precision    recall  f1-score   support

           0       0.79      0.74      0.77        31
           1       0.80      0.84      0.82        38

    accuracy                           0.80        69
   macro avg       0.80      0.79      0.79        69
weighted avg       0.80      0.80      0.80        69

svm:
               precision    recall  f1-score   support

           0       0.69      0.35      0.47        31
           1       0.62      0.87      0.73        38

    accuracy                           0.64        69
   macro avg       0.66      0.61      0.60        69
weighted avg       0.65      0.64      0.61        69

rf:
               precision    recall  f1-score   support

           0       0.89      0.77      0.83        31
           1       0.83      0.92      0.88        38

    accuracy                           0.86        69
   macro avg       0.86      0.85      0.85        69
weighted avg       0.86      0.86      0.85        69

GBD

In [16]:
esb_clf=VotingClassifier(estimators=[
    ('rf_clf',rf_clf),
    ('svm_clf',SVC(probability=True)),
    ('lr_clf',lr_clf),
    ('GBDT',gbr),
    ('AdaBoost',bdt),
    ('Xgboost',xgb)
    ],
    weights=[0.4,0.2,0.3,0.35,1,0.3],
    voting='soft')
esb_clf2=VotingClassifier(estimators=[
    ('rf_clf',rf_clf),
    ('svm_clf',SVC(probability=True)),
    ('lr_clf',lr_clf),
    ('GBDT',gbr),
    ('AdaBoost',bdt),
    ('Xgboost',xgb)],
    weights=[0.4,0.2,0.3,0.35,1,0.3],
    voting='hard')

In [17]:
esb_clf.fit(X_train,y_train)
esb_pred=esb_clf.predict(X_test)
esb_clf2.fit(X_train,y_train)
esb_pred2=esb_clf2.predict(X_test)

E:\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [18]:
print('ensemble-soft:\n',report(y_test,esb_pred))
print('ensemble-hard:\n',report(y_test,esb_pred2))

ensemble-soft:
               precision    recall  f1-score   support

           0       0.86      0.77      0.81        31
           1       0.83      0.89      0.86        38

    accuracy                           0.84        69
   macro avg       0.84      0.83      0.84        69
weighted avg       0.84      0.84      0.84        69

ensemble-hard:
               precision    recall  f1-score   support

           0       0.93      0.84      0.88        31
           1       0.88      0.95      0.91        38

    accuracy                           0.90        69
   macro avg       0.90      0.89      0.90        69
weighted avg       0.90      0.90      0.90        69

